In [1]:
# Client Setup
from dotenv import load_dotenv
import voyageai

load_dotenv()

client = voyageai.Client()

/Users/stevenwang/learning_anthropic/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Chunk by section
import re


def chunk_by_section(document_text):
    pattern = r"\n## "
    return re.split(pattern, document_text)

In [3]:
# Embedding Generation
def generate_embedding(chunks, model="voyage-3-large", input_type="query"):
    is_list = isinstance(chunks, list)
    input = chunks if is_list else [chunks]
    result = client.embed(input, model=model, input_type=input_type)
    return result.embeddings if is_list else result.embeddings[0]

In [ ]:
# VectorIndex implementation
import math
from typing import Optional, Any, List, Dict, Tuple


class VectorIndex:
    def __init__(
        self,
        distance_metric: str = "cosine",
        embedding_fn=None,
    ):
        self.vectors: List[List[float]] = []
        self.documents: List[Dict[str, Any]] = []
        self._vector_dim: Optional[int] = None
        if distance_metric not in ["cosine", "euclidean"]:
            raise ValueError("distance_metric must be 'cosine' or 'euclidean'")
        self._distance_metric = distance_metric
        self._embedding_fn = embedding_fn

    def add_document(self, document: Dict[str, Any]):
        if not self._embedding_fn:
            raise ValueError(
                "Embedding function not provided during initialization."
            )
        if not isinstance(document, dict):
            raise TypeError("Document must be a dictionary.")
        if "content" not in document:
            raise ValueError(
                "Document dictionary must contain a 'content' key."
            )

        content = document["content"]
        if not isinstance(content, str):
            raise TypeError("Document 'content' must be a string.")

        vector = self._embedding_fn(content)
        self.add_vector(vector=vector, document=document)

    def search(
        self, query: Any, k: int = 1
    ) -> List[Tuple[Dict[str, Any], float]]:
        """
        Search the index for the k most relevant documents to the query.
        
        Args:
            query (Any): The query.
            k (int): The number of results to return.
            
        Returns:
            List[Tuple[Dict[str, Any], float]]: A list of tuples, where each tuple contains a document and a score.
        """
        if not self.vectors:
            return []

        if isinstance(query, str):
            if not self._embedding_fn:
                raise ValueError(
                    "Embedding function not provided for string query."
                )
            query_vector = self._embedding_fn(query)
        elif isinstance(query, list) and all(
            isinstance(x, (int, float)) for x in query
        ):
            query_vector = query
        else:
            raise TypeError(
                "Query must be either a string or a list of numbers."
            )

        if self._vector_dim is None:
            return []

        if len(query_vector) != self._vector_dim:
            raise ValueError(
                f"Query vector dimension mismatch. Expected {self._vector_dim}, got {len(query_vector)}"
            )

        if k <= 0:
            raise ValueError("k must be a positive integer.")

        if self._distance_metric == "cosine":
            dist_func = self._cosine_distance
        else:
            dist_func = self._euclidean_distance

        distances = []
        for i, stored_vector in enumerate(self.vectors):
            distance = dist_func(query_vector, stored_vector)
            distances.append((distance, self.documents[i]))

        distances.sort(key=lambda item: item[0])

        return [(doc, dist) for dist, doc in distances[:k]]

    def add_vector(self, vector: List[float], document: Dict[str, Any]) -> None:
        """
        Add a vector to the index (i.e. vector store).

        Args:
            vector (list): A list of floats representing the vector.
            document (dict): A dictionary containing the document. e.g. {"content": "..."}

        Raises:
            TypeError: Vector must be a list of numbers.
            TypeError: Document must be a dictionary.
            ValueError: Document dictionary must contain a 'content' key.
            ValueError: Inconsistent vector dimension. Expected {self._vector_dim}, got {len(vector)}
        """
        if not isinstance(vector, list) or not all(
            isinstance(x, (int, float)) for x in vector
        ):
            raise TypeError("Vector must be a list of numbers.")
        if not isinstance(document, dict):
            raise TypeError("Document must be a dictionary.")
        if "content" not in document:
            raise ValueError(
                "Document dictionary must contain a 'content' key."
            )

        if not self.vectors:
            self._vector_dim = len(vector)
        elif len(vector) != self._vector_dim:
            raise ValueError(
                f"Inconsistent vector dimension. Expected {self._vector_dim}, got {len(vector)}"
            )

        self.vectors.append(list(vector))
        self.documents.append(document)

    def _euclidean_distance(
        self, vec1: List[float], vec2: List[float]
    ) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")
        return math.sqrt(sum((p - q) ** 2 for p, q in zip(vec1, vec2)))

    def _dot_product(self, vec1: List[float], vec2: List[float]) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")
        return sum(p * q for p, q in zip(vec1, vec2))

    def _magnitude(self, vec: List[float]) -> float:
        return math.sqrt(sum(x * x for x in vec))

    def _cosine_distance(self, vec1: List[float], vec2: List[float]) -> float:
        if len(vec1) != len(vec2):
            raise ValueError("Vectors must have the same dimension")

        mag1 = self._magnitude(vec1)
        mag2 = self._magnitude(vec2)

        if mag1 == 0 and mag2 == 0:
            return 0.0
        elif mag1 == 0 or mag2 == 0:
            return 1.0

        dot_prod = self._dot_product(vec1, vec2)
        cosine_similarity = dot_prod / (mag1 * mag2)
        cosine_similarity = max(-1.0, min(1.0, cosine_similarity))

        return 1.0 - cosine_similarity

    def __len__(self) -> int:
        return len(self.vectors)

    def __repr__(self) -> str:
        has_embed_fn = "Yes" if self._embedding_fn else "No"
        return f"VectorIndex(count={len(self)}, dim={self._vector_dim}, metric='{self._distance_metric}', has_embedding_fn='{has_embed_fn}')"

In [5]:
with open("./report.md", "r") as f:
    text = f.read()

In [6]:
# 1. Chunk the text by section
chunks = chunk_by_section(text)

In [7]:
# 2. Generate embeddings for each chunk
embeddings = generate_embedding(chunks)

In [8]:
# 3. Create a vector store and add each embedding to it
# Note: converted to a bulk operation to avoid rate limiting errors from VoyageAI
store = VectorIndex()

# store.add_document([{"content": chunk} for chunk in chunks])

for chunk, embedding in zip(chunks, embeddings):
    store.add_vector(embedding, {"content": chunk})


In [12]:
# 4. Some time later, a user will ask a question. Generate an embedding for it
question = "What is the main idea of the report?"
question_embedding = generate_embedding(question)


In [13]:
# 5. Search the store with the embedding, find the 2 most relevant chunks
search_results = store.search(question_embedding, k=2)
search_results

[({'content': 'Methodology\n\nThe insights compiled within this Annual Interdisciplinary Research Review represent a synthesis of findings drawn from standard departmental reporting cycles, specialized project updates, and cross-functional review meetings conducted throughout the year. Data sources included internal project databases, laboratory notebooks, financial reporting systems, legal case summaries, security incident logs, and minutes from dedicated working groups. A central review committee, comprising representatives nominated by each division head, was tasked with identifying key developments and potential cross-domain implications. This committee utilized a standardized reporting template to capture essential details, including unique identifiers (project codes, error numbers, case references, etc.) and progress metrics. Subsequent analysis focused on identifying thematic overlaps, shared challenges, and opportunities for synergistic development, forming the basis of this co